# CRAWLER AND SCRAPER OF ACM DIGITAL LIBRARY

## NEEDED LIBRARIES

In [ ]:
import pandas as pd
import cfscrape
from lxml import etree

## SET OF KEYWORDS

In [ ]:
keywords = ['DDoS attack', 'denial of service','DDoS']

## INSTANTIATING THE CRAWLER AND DEFINING AN REALISTIC HEADER 

In [ ]:
scraper = cfscrape.create_scraper()

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'}

## READING OLD DATASET FOR APPENDING or CREATE A NEW (EMPTY) DATAFRAME

In [ ]:
df= pd.read_csv('20191001_acm_dl_ddos_papers.csv').drop(['Unnamed: 0'],axis=1)
# df = pd.DataFrame()

## LOOP FOR CRAWLING AND SCRAPPING (IN 2 LEVELS) ALL ENTRIES RELATED TO THE SET OF KEYWORDS 

In [ ]:
for keyword in keywords:
    page = 1 
    while True:
        print(page,keyword)
        
        keyword = keyword.replace(' ','+')
        url = "https://dl.acm.org/results.cfm?query=%22"+keyword+"%22&start="+str((page-1)*20)+"&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"
        print(url)
        
        scraped_html=scraper.get(url,headers=header).content
        html = etree.HTML(scraped_html)
        
        max_entries = html.xpath("//div[@class='pagerange']/text()")[0].split()[-1]
        if (page-1)*20 > int(max_entries):
            break
        
        df_temp = pd.DataFrame()
        
        #Crawler first level
        title = []
        links = []
        publication_date = []
        venue=[]

        for entry in html.xpath("//div[@class='title']/a"):
            try:
                title.append(entry.xpath("text()")[0])
            except:
                title.append("")

            try: 
                links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
            except:
                links.append("")

        for entry in html.xpath("//div[@class='source']"): 
            try:
                publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
            except:
                publication_date.append("")

            try:
                venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
            except:
                venue.append("")

        df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})
    
        #Crawler second level
        for link in df_temp['links']:
            print(link)
            # CHECK IF THE LINK ALREADY EXIST
            if len(df[df['links']==link]) >0:
                df_temp.drop(df_temp.loc[df_temp['links']==link].index, inplace=True)
                print("duplicated")
                continue
                
            
            scraped_html=scraper.get(link,headers=header).content
            html = etree.HTML(scraped_html)

            index_current = df_temp.index[df_temp['links'] == link][0]

            try:
                df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
            except:
                df_temp.at[index_current, 'link4download'] = ""

            try:
                df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
            except:
                df_temp.at[index_current, 'num_citations'] =""

            try:
                df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
            except:
                df_temp.at[index_current, 'num_downloads_cu'] =""

            try:
                df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
            except:
                try: 
                    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
                except:
                    try:
                        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
                    except:
                        df_temp.at[index_current, 'abstract'] = ""
        
        df = df.append(df_temp,ignore_index=True)
        display(df.tail(1))
        
        page = page + 1

In [ ]:
len(df)

## SAVING FINDINGS TO PROCESS OFFLINE

In [ ]:
df.to_csv('20191001_acm_dl_ddos_papers.csv')

## DEBUGGING ERROS 1ST LEVEL CRAWLER

In [ ]:
url="https://dl.acm.org/results.cfm?query=%22denial+of+service%22&start=300&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"

In [ ]:
scraped_html=scraper.get(url,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
title = []
links = []
publication_date = []
venue=[]

for entry in html.xpath("//div[@class='title']/a"):
    try:
        title.append(entry.xpath("text()")[0])
    except:
        title.append("")
    
    try: 
        links.append('https://dl.acm.org/'+entry.xpath("@href")[0]+'&preflayout=flat')
    except:
        links.append("")

for entry in html.xpath("//div[@class='source']"): 
    try:
        publication_date.append(entry.xpath("span[@class='publicationDate']/text()")[0])
    except:
        publication_date.append("")
    
    try:
        venue.append(entry.xpath("span[@style='padding-left:10px']/text()")[0])
    except:
        venue.append("")

df_temp = pd.DataFrame({'title': title,'links': links,'publication_date': publication_date,'venue': venue,})

## DEBUGING ERROS 2nd LEVEL CRAWLER

In [ ]:
link = "https://dl.acm.org/citation.cfm?id=372148&preflayout=flat"

In [ ]:
scraped_html=scraper.get(link,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
index_current = 0

try:
    df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
except:
    df_temp.at[index_current, 'link4download'] = ""

try:
    df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
except:
    df_temp.at[index_current, 'num_citations'] =""

try:
    df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
except:
    df_temp.at[index_current, 'num_downloads_cu'] =""

try:
    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
except:
    try: 
        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
    except:
        try:
            df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
        except:
            df_temp.at[index_current, 'abstract'] = ""

df_temp

<h2 align='center'>###############################################################<br>###############################################################<br>OFFLINE PROCESSING!!!</h2>

## READING THE DATAFRAME 

In [ ]:
df= pd.read_csv('20191001_acm_dl_ddos_papers.csv').drop(['Unnamed: 0'],axis=1)

## QUICK PREVIEW OF THE DATA

In [ ]:
df.head()

## ISOLATING THE ID IN A COLUMN

In [ ]:
df['id'] = df['links'].apply(lambda x: x.replace("https://dl.acm.org/citation.cfm?id=","").replace("&preflayout=flat",""))

## CREATING SOME EMPTY COLUMNS FOR MANUAL CLASSIFICATION

In [ ]:
df['dm']=""
df['attack_type']= ""
df['defence_deployment']=""
df['not_attack_type']=""
df['attack_impact']=""
df['attack_infra']=""
df['overview']= ""

## EXPORTING COLUMNS TO CSV FOR MANUAL CLASSIFICATION

In [ ]:
df[['id','abstract','overview','dm','attack_type','defence_deployment','attack_infra','attack_impact','not_attack_type']].to_excel('abstracts4classifying.xlsx')